In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data Reading

In [2]:
import pandas as pd
pd.set_option('display.max_colwidth',100)

In [5]:
Data=pd.read_excel('drive/My Drive/Summer Internship 2020 July/My Test File/Ankit/2020 Dataset/new data/hasoc_2020_hi_train.xlsx',sep='\t')
Data.head()

,tweet_id,text,task1,task2,ID
0,1127755185944711170,"1 आदमीं को मारने पर गोडसे आतंकी हो सके है तो\n17000 सिखो, 5000 भोपाली, 3000 तमिलों का कत्लेआम कर...",HOF,HATE,hasoc_2020_hi_2250
1,1123578938406592513,"RT @Vishesh4: @jawaharyadavbjp जवाहर यादव, अगर हिम्मत है तो पूरा वीडियो डालो फिर तुम्हारे झूठ सा...",NOT,NONE,hasoc_2020_hi_381
2,1127750010156642304,RT @FunKeyBaat: #भगवा वस्त्र पहन कर मतदान नही कर सकते लेकिन बुरखा पहन के कर सकते हैं @yadavakhil...,HOF,HATE,hasoc_2020_hi_1510
3,1127660759553597441,Yey nina khothani labafazi benu phambili Finish and klaar 🍑🌊,HOF,PRFN,hasoc_2020_hi_1010
4,1123487255136489472,RT @Rajeshbhanjan2: जब भी कोई सिकुलर कोंग्रेसी गुलाम चाटुकार चरणचाट चमचा 'गाय' और 'बछड़े' को लेक...,HOF,HATE,hasoc_2020_hi_331


# Data Preprocessing
Stop Word Removal

In [6]:
#rem0ving emoji

import re

def deEmojify(txt):
    regrex_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',txt)

Data['text']=Data['text'].apply(lambda x:deEmojify(x))
Data[['text','task1']].head()

,text,task1
0,"1 आदमीं को मारने पर गोडसे आतंकी हो सके है तो\n17000 सिखो, 5000 भोपाली, 3000 तमिलों का कत्लेआम कर...",HOF
1,"RT @Vishesh4: @jawaharyadavbjp जवाहर यादव, अगर हिम्मत है तो पूरा वीडियो डालो फिर तुम्हारे झूठ सा...",NOT
2,RT @FunKeyBaat: #भगवा वस्त्र पहन कर मतदान नही कर सकते लेकिन बुरखा पहन के कर सकते हैं @yadavakhil...,HOF
3,Yey nina khothani labafazi benu phambili Finish and klaar,HOF
4,RT @Rajeshbhanjan2: जब भी कोई सिकुलर कोंग्रेसी गुलाम चाटुकार चरणचाट चमचा 'गाय' और 'बछड़े' को लेक...,HOF


Removing URL

In [7]:
# remove URL
def remove_URL(txt):
    url= re.compile(r"https?://\S+|www\.\S+")
    return url.sub(r"",txt)

Data['text']=Data['text'].apply(lambda x:remove_URL(x))
Data[['text']].head()

Data['text']=Data['text'].str.replace(r'\d+','')
Data[['text','task1']].head()

,text,task1
0,"आदमीं को मारने पर गोडसे आतंकी हो सके है तो\n सिखो, भोपाली, तमिलों का कत्लेआम करवाने वाला भारत...",HOF
1,"RT @Vishesh: @jawaharyadavbjp जवाहर यादव, अगर हिम्मत है तो पूरा वीडियो डालो फिर तुम्हारे झूठ साम...",NOT
2,RT @FunKeyBaat: #भगवा वस्त्र पहन कर मतदान नही कर सकते लेकिन बुरखा पहन के कर सकते हैं @yadavakhil...,HOF
3,Yey nina khothani labafazi benu phambili Finish and klaar,HOF
4,RT @Rajeshbhanjan: जब भी कोई सिकुलर कोंग्रेसी गुलाम चाटुकार चरणचाट चमचा 'गाय' और 'बछड़े' को लेकर...,HOF


# Tokenization, Encoding, padding after preprocessing

In [8]:
import keras
from keras.preprocessing.text import Tokenizer
tok1 = Tokenizer(char_level=False, filters ='!"$%&@()*+,-./:;”“<=>?[\\]^_`{|}~\t\n', lower = True)

In [9]:
tok1.fit_on_texts(Data['text'])
words=len(tok1.word_counts)
words

13545

In [10]:
word_list=[tok1.word_index]
#word_list

In [11]:
encoded=tok1.texts_to_sequences(Data['text'])
#print(Data['text'][0])
#encoded[1211]

In [12]:
##         Padding encoded sequence of words
from keras.preprocessing import sequence
max_length=20
padd = sequence.pad_sequences(encoded, maxlen=max_length, padding='post')
#padd[0:12]

# Label Encoding

In [13]:
import numpy as np
from keras.utils.np_utils import to_categorical
from sklearn import preprocessing
labelEncode=preprocessing.LabelEncoder()
labelEncode.fit(Data['task1'])
print (labelEncode.classes_)
train_labelEncode=labelEncode.transform(Data['task1'])
train_label=to_categorical(np.asarray(train_labelEncode))

['HOF' 'NOT']


#Importing libraries for Model creation

In [43]:
# importing libraries for creating model
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Embedding,MaxPool1D
from keras.layers import Conv1D, GlobalMaxPooling1D, MaxPooling1D

# CNN model

In [44]:
model = Sequential()
model.add(Embedding(input_dim = words + 1, output_dim = 100,input_length=max_length))
model.add(Conv1D(filters =512,kernel_size=2, activation='relu'))
model.add(MaxPooling1D(3))
model.add(Flatten())
model.add(Dense(128, activation = 'relu'))
model.add(Dense(2, activation = 'softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 100)           1354600   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 19, 512)           102912    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 6, 512)            0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 3072)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               393344    
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 258       
Total params: 1,851,114
Trainable params: 1,851,114
Non-trainable params: 0
____________________________________________

# Model Compilation

In [45]:
## compile

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

# Model Training

In [46]:
model.fit(padd,train_label,epochs=15,verbose=1,batch_size= 32)

Epoch 1/15
93/93 [==============================] - 1s 14ms/step - loss: 0.5984 - acc: 0.7067
Epoch 2/15
93/93 [==============================] - 1s 14ms/step - loss: 0.3621 - acc: 0.8296
Epoch 3/15
93/93 [==============================] - 1s 14ms/step - loss: 0.0620 - acc: 0.9862
Epoch 4/15
93/93 [==============================] - 1s 14ms/step - loss: 0.0283 - acc: 0.9960
Epoch 5/15
93/93 [==============================] - 1s 13ms/step - loss: 0.0113 - acc: 0.9980
Epoch 6/15
93/93 [==============================] - 1s 14ms/step - loss: 0.0087 - acc: 0.9983
Epoch 7/15
93/93 [==============================] - 1s 13ms/step - loss: 0.0073 - acc: 0.9983
Epoch 8/15
93/93 [==============================] - 1s 13ms/step - loss: 0.0075 - acc: 0.9983
Epoch 9/15
93/93 [==============================] - 1s 13ms/step - loss: 0.0063 - acc: 0.9983
Epoch 10/15
93/93 [==============================] - 1s 13ms/step - loss: 0.0058 - acc: 0.9987
Epoch 11/15
93/93 [==============================] - 1s 13m

#Test data reading

In [47]:
Test =pd.read_csv('drive/My Drive/Summer Internship 2020 July/My Test File/Ankit/2020 Dataset/test data/hindi_test_1509.csv')
Test.dropna(inplace = True)
Test.head()

,tweet_id,text,task1,task2,ID
0,1127781647783301120,@Kinjal_Dubey_ इसके लिए तैयारी हो रखी है\nआपको जान देने की जरूरत नहीं है।\nरक्षा मंत्री एक महिला...,HOF,OFFN,hasoc_2020_hi_1938
1,1127492853171535872,"RT @pratimamishra04: दिल्ली के मुख्यमंत्री @ArvindKejriwal ने अपने पूरे परिवार के साथ वोट डाला, ...",NOT,NONE,hasoc_2020_hi_1201
2,1123638375896903681,"RT @nishantdabre2: आज एक तरफ खुशी है, तो एक तरफ दुख\nएक तरफ मसूद अजहर को पूरे विश्व के आतंकी के ...",NOT,NONE,hasoc_2020_hi_1628
3,1127496460310683648,@BJP4India @HardeepSPuri जिस कुनबे के लोग :-\n12वीं पास डिग्री छुपाने वाली को शिक्षा मंत्री बनात...,NOT,NONE,hasoc_2020_hi_3257
4,1127667277535399937,@manakgupta बादल....😂😂😂,NOT,NONE,hasoc_2020_hi_4133


# Test data preprocessing

In [48]:
## doing preprocessing as above
Test['text']=Test['text'].apply(lambda x:deEmojify(x))
Test['text']=Test['text'].apply(lambda x: remove_URL(x))
Test['text']=Test['text'].str.replace(r'\d+','')
t= Test['text']

# Test data encoding, padding

In [49]:
encoded1 =tok1.texts_to_sequences(t)

In [50]:
padded = sequence.pad_sequences(encoded1, maxlen=max_length, padding='post')
padded[0]

array([  1, 289, 660, 217,   5, 746,  11,  28, 656, 880,  23, 719,   1,
        35,  68,  12, 661,   6, 975,  11], dtype=int32)

# Test label encoding

In [51]:
test_labelEncode=labelEncode.transform(Test['task1'])
test_label=to_categorical(np.asarray(test_labelEncode))

# Test data Model prediction

In [52]:
test_predictions=model.predict(padded)
test_predictions2=np.argmax(test_predictions,axis=1)

# Test data Classification report

In [53]:
#Accuracy 
from sklearn.metrics import classification_report
print(classification_report(test_labelEncode,test_predictions2))

              precision    recall  f1-score   support

           0       0.62      0.41      0.50       197
           1       0.78      0.89      0.83       466

    accuracy                           0.75       663
   macro avg       0.70      0.65      0.67       663
weighted avg       0.74      0.75      0.73       663



# Adding predicted lable as a colum to dataFrame 'Test'

In [54]:
Test['Predictionlabel']=labelEncode.inverse_transform(test_predictions2)
Test.head()

,tweet_id,text,task1,task2,ID,Predictionlabel
0,1127781647783301120,@Kinjal_Dubey_ इसके लिए तैयारी हो रखी है\nआपको जान देने की जरूरत नहीं है।\nरक्षा मंत्री एक महिला...,HOF,OFFN,hasoc_2020_hi_1938,NOT
1,1127492853171535872,"RT @pratimamishra: दिल्ली के मुख्यमंत्री @ArvindKejriwal ने अपने पूरे परिवार के साथ वोट डाला, कह...",NOT,NONE,hasoc_2020_hi_1201,NOT
2,1123638375896903681,"RT @nishantdabre: आज एक तरफ खुशी है, तो एक तरफ दुख\nएक तरफ मसूद अजहर को पूरे विश्व के आतंकी के र...",NOT,NONE,hasoc_2020_hi_1628,NOT
3,1127496460310683648,@BJPIndia @HardeepSPuri जिस कुनबे के लोग :-\nवीं पास डिग्री छुपाने वाली को शिक्षा मंत्री बनाते ह...,NOT,NONE,hasoc_2020_hi_3257,NOT
4,1127667277535399937,@manakgupta बादल....,NOT,NONE,hasoc_2020_hi_4133,NOT


# Generating CSV file

In [55]:
HindiTask1File=Test[['tweet_id','Predictionlabel','ID']]
HindiTask1File.head()
HindiTask1File.to_csv('submission_HI_A.csv',index=False)